In [1]:
import pandas as pd
import numpy as np
import random

E:\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
E:\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Here my idea

#### below visible code needed for defining possible cards and a deck

In [ ]:
list1 = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
list2 = ["Spade", "Club", "Dimond", "Heart"]

In [ ]:
len(list1)
len(list2)

In [2]:
possible_cards = ("Spade 2","Spade 3","Spade 4","Spade 5","Spade 6","Spade 7","Spade 8","Spade 9","Spade 10","Spade J","Spade Q","Spade K","Spade A","Club 2","Club 3","Club 4","Club 5","Club 6","Club 7","Club 8","Club 9","Club 10","Club J","Club Q","Club K","Club A","Dimond 2","Dimond 3","Dimond 4","Dimond 5","Dimond 6","Dimond 7","Dimond 8","Dimond 9","Dimond 10","Dimond J","Dimond Q","Dimond K","Dimond A","Heart 2","Heart 3","Heart 4","Heart 5","Heart 6","Heart 7","Heart 8","Heart 9","Heart 10","Heart J","Heart Q","Heart K","Heart A")

In [3]:
print(random.choice(possible_cards))

Club J


In [4]:
starting_deck = list(possible_cards)

In [5]:
#dictionary to determine carfds value for game purposes
#Aces dual value omitted - assigned to 11 for now, to be reflected when calculating cards value
cards_value = {
    'Spade 2' :2,
    'Spade 3' :3,
    'Spade 4' :4,
    'Spade 5' :5,
    'Spade 6' :6,
    'Spade 7' :7,
    'Spade 8' :8,
    'Spade 9' :9,
    'Spade 10' : 10,
    'Spade J' : 10,
    'Spade Q' :10,
    'Spade K' :10,
    'Spade A' :11,
    'Club 2' :2,
    'Club 3' :3,
    'Club 4' :4,
    'Club 5' :5,
    'Club 6' :6,
    'Club 7' :7,
    'Club 8' :8,
    'Club 9' :9,
    'Club 10' :10,
    'Club J' : 10,
    'Club Q' : 10,
    'Club K' : 10,
    'Club A' : 11,
    'Dimond 2' :2,
    'Dimond 3' :3,
    'Dimond 4' :4,
    'Dimond 5' :5,
    'Dimond 6' :6,
    'Dimond 7' :7,
    'Dimond 8' :8,
    'Dimond 9' :9,
    'Dimond 10' :10,
    'Dimond J' : 10,
    'Dimond Q' : 10,
    'Dimond K' : 10,
    'Dimond A' : 11,
    'Heart 2' :2,
    'Heart 3' :3,
    'Heart 4' :4,
    'Heart 5' :5,
    'Heart 6' :6,
    'Heart 7' :7,
    'Heart 8' :8,
    'Heart 9' :9,
    'Heart 10' :10,
    'Heart J' : 10,
    'Heart Q' : 10,
    'Heart K' : 10,
    'Heart A' : 11}

In [8]:
def shuffle(decks):
    #building a deck
    global deck
    deck = random.sample(starting_deck, len(starting_deck))
    for i in range(decks - 1):
        deck += random.sample(starting_deck, len(starting_deck))
    return deck

In [9]:
#testing the results of function shuffle
deck = shuffle(decks=6)
print(deck)
print(deck.count("Heart A"))

['Heart 6', 'Heart 2', 'Dimond 6', 'Dimond 10', 'Club Q', 'Spade 2', 'Club K', 'Dimond 3', 'Spade Q', 'Spade J', 'Heart 9', 'Dimond J', 'Heart A', 'Club 6', 'Club 7', 'Heart J', 'Dimond 9', 'Dimond 5', 'Heart 5', 'Club 8', 'Spade 9', 'Heart K', 'Club 5', 'Heart 4', 'Dimond 4', 'Dimond Q', 'Club J', 'Spade 8', 'Dimond 2', 'Heart 10', 'Club 4', 'Club 9', 'Spade 4', 'Spade 6', 'Spade 5', 'Heart 7', 'Heart 8', 'Club 3', 'Dimond A', 'Spade A', 'Spade 7', 'Club A', 'Heart 3', 'Dimond 7', 'Club 10', 'Dimond K', 'Club 2', 'Spade 3', 'Heart Q', 'Dimond 8', 'Spade K', 'Spade 10', 'Club 5', 'Dimond 6', 'Club 10', 'Club 4', 'Heart 2', 'Club J', 'Club 8', 'Club 9', 'Spade 9', 'Club 3', 'Dimond 10', 'Spade 7', 'Dimond J', 'Spade K', 'Dimond 8', 'Heart Q', 'Heart A', 'Club K', 'Dimond 2', 'Spade A', 'Spade 2', 'Heart 6', 'Dimond 3', 'Heart 4', 'Heart 3', 'Club 6', 'Dimond Q', 'Dimond 5', 'Dimond 4', 'Spade 5', 'Spade 4', 'Club 7', 'Heart 7', 'Club 2', 'Spade 3', 'Spade 6', 'Spade 10', 'Heart 5', 'Dim

In [ ]:
rd=random.sample(starting_deck, len(starting_deck))
for card in rd:
    print(card)

In [36]:
#define the outcome of 1 BJ game (1 player vs casino)
#nececarry to place here the mechanics of Aces (soft/hard), for now Aces are set to 11

#currently the function is in working status, not optimized for simulation, but for fixing bugs


def game_of_blackjack(players_engine="rand"):
    global deck
    def get_value(hand):
        #hand should be a list argument
        
        #calculate value and adjust for aces
        value = 0
        for card in hand:
            value += cards_value[card]
            
        aces = 0
        for card in hand: 
            if card[-1] == "A":
                aces += 1

        while value > 21 and aces >0:
            aces -= 1
            value -= 10
        return value

               
    def check_for_bust(played_hand):
        value_hand = get_value(played_hand)
        if value_hand > 21:
            print("Casino wins")
            lost_hands.append(played_hand)
            active_hands.remove(played_hand)
            return False
        else:
            #results = df.append(new_row, ignore_index=True)
            return True


    def split(played_hand):
        if len(played_hand) != 2 or played_hand[0] != played_hand[1]:
            return False
        hand2 = [played_hand[1]]
        played_hand.pop(1)
        played_hand += [deck[0]]
        deck.pop(0)
        active_hands.append(hand2)
        return

    
    def double(played_hand):
        if len(played_hand) != 2:
            return False
        played_hand += [deck[0]]
        deck.pop(0)
        check_for_bust(played_hand)
        stand(played_hand)
        return

    
    def hit(played_hand):
        played_hand.append(deck[0])
        deck.pop(0)
        check_for_bust(played_hand)
        return

    def stand(played_hand):
        played_hands.append(played_hand)
        active_hands.remove(played_hand)
        return

    def casino_move():
        #implement an after split scenario for multiple hands
        dealers_cards.append(deck[0])
        deck.pop(0)
        value_dealer = get_value(dealers_cards)
    
        if value_dealer == 21:
            #a scenario of dealers instatnt blackjack is dealt here
            print("Casino wins")
            for hand in played_hands:
                lost_hands.append(hand)
                played_hands.remove(hand)
                        
        while value_dealer < 17:
            dealers_cards.append(deck[0])
            deck.pop(0)
            value_dealer = get_value(dealers_cards)
            if value_dealer > 21:
                for hand in list(played_hands):
                    print("Player wins")
                    won_hands.append(hand)
                    played_hands.remove(hand)
                            
        for hand in list(played_hands):
            if value_dealer == get_value(hand):
                print("Push")
                pushed_hands.append(hand)
                played_hands.remove(hand)
            elif value_dealer > get_value(hand):
                print("Casino wins")
                lost_hands.append(hand)
                played_hands.remove(hand)
            else:
                print("Player wins")
                won_hands.append(hand)
                played_hands.remove(hand)
        return



    print("Game starts...")
    #drawing cards
    players_cards = [deck[0]]
    deck.pop(0)
    players_cards += [deck[0]]
    deck.pop(0)
    dealers_cards = [deck[0]]
    deck.pop(0)
    active_hands = [players_cards] #list of active players' hands
    played_hands = [] #list of inactive players' hands
    lost_hands = [] #list of hands that lost
    won_hands = [] #list of hands that won
    pushed_hands = []
    #checking for BlackJack
    if get_value(players_cards) == 21:
        print("Player wins")
        won_hands.append(players_cards)
        active_hands.remove(players_cards)
        #results = df.append(new_row, ignore_index=True)
    print(players_cards)
    print(dealers_cards)
        
        
        
    #players decision engine
    #in future there should be rand (random choices), basic (BJ basic strategy - proven best possible move)
    #if players_engine == "rand":
    #lets just assume its the only engine
    moves = [stand, hit, double, split]
    print("Active hands:  ")
    print(active_hands)
    while True:
        #played_hand = active_hands[0]
        #if len(played_hand) == 1:
            #hit(played_hand)
        move = random.choice(moves)
        print(move)
        if move == "stand":
            print("player standing")
            stand(active_hands[0])
        if len(active_hands)==0:
            break            
        if move == "hit":
            print("player hitting")
            hit(active_hands[0])
        if len(active_hands)==0:
            break
        if move == "double":
            print("player doubling")
            double(active_hands[0])
        if len(active_hands)==0:
            break
        if move == "split":
            print("split")
            split(played_hand)
        if len(active_hands)==0:
            break
    casino_move()
        
        
    

## Developement plan

1. Make the random function work (make sure the arguments inside local functions work properly
2. Make Basic Strategy engine
3. Makee it appened to data frame
3. Make bets - possible outcomes
3. Make "miss engine" - one that every one in a while misses the best move
1. Make exceptions engine - to take into account true count into play
1. optimize????

In [37]:
shuffle(6)

['Heart 10',
 'Club 7',
 'Spade Q',
 'Dimond Q',
 'Dimond 10',
 'Spade 10',
 'Spade 2',
 'Spade 3',
 'Club 5',
 'Dimond 9',
 'Club 9',
 'Dimond 3',
 'Club A',
 'Heart 6',
 'Heart 4',
 'Club 8',
 'Dimond 5',
 'Heart J',
 'Dimond 2',
 'Club Q',
 'Club 2',
 'Heart 7',
 'Spade 7',
 'Dimond 6',
 'Spade A',
 'Spade 4',
 'Dimond 4',
 'Dimond A',
 'Heart 5',
 'Heart K',
 'Club 4',
 'Spade 6',
 'Heart Q',
 'Spade 5',
 'Club K',
 'Spade 8',
 'Dimond 7',
 'Heart A',
 'Club J',
 'Dimond K',
 'Spade 9',
 'Dimond 8',
 'Dimond J',
 'Spade K',
 'Heart 3',
 'Club 6',
 'Club 3',
 'Club 10',
 'Heart 9',
 'Heart 8',
 'Spade J',
 'Heart 2',
 'Club J',
 'Club 7',
 'Dimond A',
 'Dimond 4',
 'Heart 2',
 'Heart 10',
 'Spade 9',
 'Club 8',
 'Spade 8',
 'Heart J',
 'Spade 10',
 'Club 4',
 'Spade J',
 'Dimond 2',
 'Club K',
 'Heart K',
 'Club 2',
 'Dimond 3',
 'Dimond 10',
 'Heart 6',
 'Club 10',
 'Dimond 9',
 'Dimond 6',
 'Club 6',
 'Spade 6',
 'Dimond K',
 'Spade Q',
 'Dimond Q',
 'Heart 8',
 'Spade 2',
 'Heart

In [38]:
game_of_blackjack()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<function game_of_blackjack.<locals>.hit at 0x00000287DF916700>
<function game_of_blackjack.<locals>.hit at 0x00000287DF916700>
<function game_of_blackjack.<locals>.split at 0x00000287DF95B820>
<function game_of_blackjack.<locals>.hit at 0x00000287DF916700>
<function game_of_blackjack.<locals>.split at 0x00000287DF95B820>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.hit at 0x00000287DF916700>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.double at 0x00000287DF916E50>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.split at 0x00000287DF95B820>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.stand at 0x00000287DF916550>
<function game_of_blackjack.<locals>.double at 0x00000287DF916E50>
<function game_o

KeyboardInterrupt: 